# Riddler Classic
https://fivethirtyeight.com/features/come-on-down-and-escape-the-maze/

From Josh Berry, come on down!

You’re playing a “Price Is Right” game called Cover Up, which has contestants try to guess all five digits of the price of a brand new car. You have two numbers to choose from for the first digit, three numbers to choose from for the second digit, and so on, ending with six options for the fifth and final digit. You’re not winning any $100,000 cars in this game.

First, you lock in a guess at the entire price of the car. If you get at least one digit correct on the first guess, the correct digit(s) are highlighted and you get to replace incorrect digits on a second guess. This continues on subsequent guesses until the price is guessed correctly. But if none of the new numbers you swapped in are correct, you lose. A contestant could conceivably win the car on the first guess or with five guesses, getting one additional correct digit highlighted on each guess.

First question: If you’re guessing entirely by chance, what’s the likelihood of winning the car?

Second question: Suppose you know a little bit about cars. Specifically, you are 100 percent certain about the digit in the ten-thousands place, but have to guess the remaining four digits by chance. What’s the best strategy, and what’s the likelihood of winning the car now?

In [1]:
import numpy as np
from copy import deepcopy

In [2]:
#Let's say the following are the potential candidates for the digits
#Note: Starting the arrays from 0 makes the subsequent code cleaner, WLOG.
digits = [[1, 2],
          [1, 2, 3],
          [1, 2, 3, 4],
          [1, 2, 3, 4, 5],
          [1, 2, 3, 4, 5, 6]
]

#Generate a random value for the car
def gen_value(digits=digits):
    return [int(np.random.choice(digits[x])) for x in range(len(digits))]


def game_round(real_price,guessed_price, d, prev_success=0, cheat = 0):

    current_success = sum([guessed_price[i] == real_price[i] for i in range(len(guessed_price))])
    
    if guessed_price == real_price:
        return 1
    
    elif current_success <= prev_success:
        return 0
    
    #Update the digit pool to choose from
    for idx, digit in enumerate(real_price):
        
        if len(d[idx]) != 1:
            #Guessed correctly -> assign the digit
            if guessed_price[idx] == digit:
                d[idx] = [digit]
            #Guessed incorrectly -> remove the digit
            else:
                d[idx].remove(guessed_price[idx])
    if cheat == 1 and d[0] != 2:
        d[0] = [2]
    new_guessed_price = gen_value(d)#Update the guessed price
    return game_round(real_price, new_guessed_price, d, current_success)


def game(digits,n=10000,cheat=0):
    success_count = 0
    for i in range(n):
        real_price = gen_value(digits) #The value of the car we're trying to guess
        d = deepcopy(digits)
        if cheat == 1:
            d[0] = [1]
        else:
            d[0] = [2]
        guessed_price = gen_value(digits) #initial guess
        
        success_count += game_round(real_price,guessed_price, d,cheat=cheat) #number of succeses

    return success_count/n

# Rigorous calculation

Start with a simpler example:
you could've found 0th or first or both:
* if we only correctly guess the 0th: 1/2 x 2/3 x 1/2 = 1/6
* if only the first, but not the 0th = 1/3 x 1/2 x 1 = 1/6
* if we correctly guess both = 1/6  
Sums up to 1/2

In [3]:

def bin_comb(k):
    end = 1<<k   #this is the integer with binary developpement 1 followed by k zeros
    result = []
    for j in range(end): # iterate until end means getting all k - 0-1 combinations
        comb = bin(j)[2:].zfill(k)
        result.append([int(x) for x in comb])
    return result

def guess(l=[2,3]):
    combs = bin_comb(len(l)) #for len=2: [[0,0],[1,0],[0,1],[1,1]]
    pr=0
    #Remove the list with all zeros from combs
    combs = combs[1:]
    for idn, i in enumerate(l):
        if i == 1:
            combs = [x for x in combs if x[idn] != 0]
            
    for idy, comb in enumerate(combs):
        internal_p = 1
        
        new_l = []
        for idx, dg in enumerate(comb):
            
            if dg == 0:
                internal_p *= (l[idx]-1)/(l[idx])
                new_l.append(l[idx]-1)
            else:
                internal_p *= 1/l[idx]
        if new_l != []:
            internal_p *= guess(new_l)

        pr += internal_p
    return pr
                
guess([2,3,4,5,6])

0.32083333333333325

# We Know the First Digit
As for the case where we know the first digit, I cannot think of any better strategy than saying it in the first round because we don't gain anything from not saying it. (If we got another round after a failure, then we could've used the the known first digit as leverage, and not say it immediately, but that's not the case.)

What we gain by saying it in the first round is that we learn something about the other digits for free. Even if we don't guess any of the other digits correctly, we get to eliminate potential values from each digit. If we don't say the first digit right-away we risk losing in the first round.

Suppose we dont say the first digit right away:
* The probability of failing is 2/3 x 3/4 x 4/5 x 5/6 = 1/3

An alternative to try is this:
save the first digit for:
* when you got all the other digits
* for the fifth round

[If the alternative is true, then you should mot say any digit that you've secured until you have to. But then it's going to be like trying to guess it at once]

### On the other hand
There are some cases where holding onto the first digit might be helpful. For instance if we guess all the fifth digit correctly in the first round then there is an 80% chance we will fail in the second round trying to guess the fifth one. If we held onto the first digit and only said it in the second round we'd get a free pass on the second round with 20% chance to guess correctly besides we would get to guess another time in the third round with 25% chance of winning.

I'm too tired to calculate it rigorously right now but I tweaked the simulation code so that we can simulate not giving the right answer for the first digit in the first round (the "cheat" option). And it doesn't yield a significant difference. So it seems like the best strategy is just to say te first digit outright, though I feel like I'm mistaken.


In [4]:
#Let's suppose that we know that the price of the car
#is in the 20,000's instead of 10's.
digits2 = [[2],
          [0, 1, 2],    
          [0, 1, 2, 3],
          [0, 1, 2, 3, 4],
          [0, 1, 2, 3, 4, 5]
]
game(digits2, n=1000000, cheat=0) #Cheat option disabled

0.321993

In [5]:
#Cheat option enabled
game(digits2, n=1000000, cheat=1)

0.322817